# 목차
## 1. 데이터 준비
## 2. 사용자 별 평균, 분산 feature 생성
## 3. review 데이터에 사용자 별 평균, 분산 feature 병합(user_id 기준 병합)
## 4. business 데이터 전처리
- ### 4.1. attributes 칼럼 전처리
- ### 4.2. category 칼럼 전처리
- ### 4.3. hours 칼럼 전처리
## 5. 최종 데이터셋 구축
## 6. 카테고리 별 데이터셋 split
## 7. 카테고리 별 split dataset 저장

# 1. 데이터 준비

In [ ]:
import numpy as np
import pandas as pd
import ast
import warnings
warnings.filterwarnings(action='ignore')

review_df = pd.read_json('../yelp_academic_dataset_review.json', lines=True)
review_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


# 2. user_id 별 평균, 분산 생성

In [ ]:
import pandas as pd

# Create a new dataframe with selected columns
new_df = review_df[['review_id', 'user_id', 'business_id', 'stars']].copy()
new_df

,review_id,user_id,business_id,stars
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4
...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5


In [ ]:
len(new_df['user_id'].unique())

1987929

In [ ]:
import pandas as pd
import numpy as np

# Group by user_id and calculate mean, variance, and count for 'stars'
user_grouped_df = new_df.groupby('user_id')['stars'].agg(['mean', 'var', 'count']).reset_index()

# # Add 'num_review' column
# user_grouped_df['num_review'] = user_grouped_df['count']

# Replace stdev with 0 where num_review is 1
user_grouped_df.loc[user_grouped_df['count'] == 1, 'var'] = 0

# Rename the columns
user_grouped_df.rename(columns={'count':'num_review'}, inplace=True)
user_grouped_df

,user_id,mean,var,num_review
0,---1lKK3aKOuomHnwAkAow,5.000000,0.000000,1
1,---2PmXbF47D870stH1jqA,5.000000,0.000000,28
2,---UgP94gokyCDuB5zUssA,3.636364,2.054545,11
3,---fa6ZK37T9NjkGKI4oSg,1.000000,0.000000,1
4,---r61b7EpVPkb4UVme5tA,4.000000,3.000000,5
...,...,...,...,...
1987924,zzz-M4QvkEpUWWPL9RTzLA,2.200000,2.400000,10
1987925,zzzCg-_lpyYE82TlJCdwdw,5.000000,0.000000,1
1987926,zzzGgfvrSJ4AQeKtcgocIw,3.000000,0.000000,1
1987927,zzzMBVS73g3ZJ7qL8JyhiA,1.000000,0.000000,1


# 3. review 와 user 데이터셋 user_id를 기반으로 merge

In [ ]:
# user_grouped_df = pd.read_csv('grouped_df(mean, stdev).csv')
# user_grouped_df

In [ ]:
import pandas as pd

# Assuming review_df and user_grouped_df are your pandas DataFrames

# Perform a left merge on 'user_id' column
total_df = pd.merge(review_df, user_grouped_df[['user_id', 'mean', 'var', 'num_review']], on='user_id', how='left')

# Print the resulting DataFrame
total_df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,mean,var,num_review
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,4.142857,1.238095,28
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,4.125000,2.125000,8
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,4.448980,0.710884,49
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,4.800000,0.200000,5
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,3.008264,2.808264,121
...,...,...,...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20,3.646643,2.002356,283
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10,5.000000,0.000000,6
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30,3.936306,1.495917,157
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,4.345238,0.951664,84


In [ ]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6990280 entries, 0 to 6990279
Data columns (total 12 columns):
 #   Column       Dtype         
---  ------       -----         
 0   review_id    object        
 1   user_id      object        
 2   business_id  object        
 3   stars        int64         
 4   useful       int64         
 5   funny        int64         
 6   cool         int64         
 7   text         object        
 8   date         datetime64[ns]
 9   mean         float64       
 10  var          float64       
 11  num_review   int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 693.3+ MB


In [ ]:
total_df.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
mean           0
var            0
num_review     0
dtype: int64

# 4. business_df 전처리

In [ ]:
business_df = pd.read_json('../yelp_academic_dataset_business.json', lines=True)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [ ]:
import numpy as np

business_df['name'] = business_df['name'].astype(str)
business_df['address'] = business_df['address'].astype(str)
business_df['state'] = business_df['state'].astype(str)
business_df['postal_code'] = business_df['postal_code'].astype(str)

In [ ]:
# attributes, categories, hours 칼럼에 결측치 존재.
# 각각 133602, 150243, 127123 정상 데이터 존재
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    136602 non-null  object 
 12  categories    150243 non-null  object 
 13  hours         127123 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


In [ ]:
for i in business_df.iloc[1]:
    print(i, type(i))

mpf3x-BjTdTEA3yCZrAYPw <class 'str'>
The UPS Store <class 'str'>
87 Grasso Plaza Shopping Center <class 'str'>
Affton <class 'str'>
MO <class 'str'>
63123 <class 'str'>
38.551126 <class 'numpy.float64'>
-90.335695 <class 'numpy.float64'>
3.0 <class 'numpy.float64'>
15 <class 'numpy.int64'>
1 <class 'numpy.int64'>
{'BusinessAcceptsCreditCards': 'True'} <class 'dict'>
Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services <class 'str'>
{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', 'Wednesday': '8:0-18:30', 'Thursday': '8:0-18:30', 'Friday': '8:0-18:30', 'Saturday': '8:0-14:0'} <class 'dict'>


In [ ]:
# fillna는 특정 값으로 대체.
# 빈 리스트나 빈 dict type으로 변환을 직접적으로 지원해주진 않음.

import pandas as pd
import numpy as np

# Assuming business_df is your pandas DataFrame

# Replace missing values in 'attributes' column
business_df['attributes'] = business_df['attributes'].apply(lambda x: {} if pd.isnull(x) else x)

# Replace missing values in 'categories' column -> str type이기 때문에 fillna()사용
business_df['categories'] = business_df['categories'].fillna('')

# Replace missing values in 'hours' column
business_df['hours'] = business_df['hours'].apply(lambda x: {} if pd.isnull(x) else x)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",{}
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",{}
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [ ]:
# attributes, hours 칼럼에 결측치 -> 빈 객체로 수정
# categories 칼럼에 결측치 -> '' 빈 str로 수정
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   150346 non-null  object 
 1   name          150346 non-null  object 
 2   address       150346 non-null  object 
 3   city          150346 non-null  object 
 4   state         150346 non-null  object 
 5   postal_code   150346 non-null  object 
 6   latitude      150346 non-null  float64
 7   longitude     150346 non-null  float64
 8   stars         150346 non-null  float64
 9   review_count  150346 non-null  int64  
 10  is_open       150346 non-null  int64  
 11  attributes    150346 non-null  object 
 12  categories    150346 non-null  object 
 13  hours         150346 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 16.1+ MB


## 4.1. attributes 칼럼 전처리

In [ ]:
attributes_df = pd.DataFrame(business_df['attributes'].tolist())

# nan 값은 정보입력 x -> 기본값 : False 처리
attributes_df = attributes_df.fillna(False)
attributes_df

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,1,False,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",...,False,False,False,False,False,False,False,False,False,False
4,False,True,True,False,False,True,False,False,False,"{'garage': None, 'street': None, 'validated': ...",...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,3,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,False,False,False,False,False,False,False,False,False,False
150343,False,True,False,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,True,True,4,False,None,None,False,False,"{'garage': False, 'street': False, 'validated'...",...,False,False,False,False,False,False,False,False,False,False


In [ ]:
attributes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 39 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   ByAppointmentOnly           150346 non-null  object
 1   BusinessAcceptsCreditCards  150346 non-null  object
 2   BikeParking                 150346 non-null  object
 3   RestaurantsPriceRange2      150346 non-null  object
 4   CoatCheck                   150346 non-null  object
 5   RestaurantsTakeOut          150346 non-null  object
 6   RestaurantsDelivery         150346 non-null  object
 7   Caters                      150346 non-null  object
 8   WiFi                        150346 non-null  object
 9   BusinessParking             150346 non-null  object
 10  WheelchairAccessible        150346 non-null  object
 11  HappyHour                   150346 non-null  object
 12  OutdoorSeating              150346 non-null  object
 13  HasTV                       1

In [ ]:
attributes_df = attributes_df.replace('True',True)
attributes_df = attributes_df.replace('False',False)
attributes_df = attributes_df.replace('None',None)

attributes_df

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,1,False,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",...,False,False,False,False,False,False,False,False,False,False
4,False,True,True,False,False,True,False,False,False,"{'garage': None, 'street': None, 'validated': ...",...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,3,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,False,False,False,False,False,False,False,False,False,False
150343,False,True,False,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,True,True,4,False,None,None,False,False,"{'garage': False, 'street': False, 'validated'...",...,False,False,False,False,False,False,False,False,False,False


전처리가 필요한 index열(str type의 dict이 들어있는 열)
  - 9,19,24,27,30,34,38

In [ ]:
for idx, column in enumerate(attributes_df.columns):
    print(idx, attributes_df[column].unique())

0 [True False None]
1 [False True None]
2 [False True None]
3 [False '2' '1' '3' '4' None]
4 [False True None]
5 [False True None]
6 [False True None]
7 [False True None]
8 [False "u'no'" "u'free'" "'free'" "'no'" None "u'paid'" "'paid'"]
9 [False
 "{'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}"
 "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}"
 "{'garage': None, 'street': None, 'validated': None, 'lot': True, 'valet': False}"
 None
 "{'garage': False, 'street': False, 'validated': False, 'lot': False, 'valet': False}"
 "{'garage': None, 'street': False, 'validated': None, 'lot': True, 'valet': False}"
 "{u'valet': False, u'garage': None, u'street': True, u'lot': False, u'validated': None}"
 "{'garage': False, 'street': True, 'validated': False, 'lot': True, 'valet': False}"
 "{'garage': True, 'street': False, 'validated': False, 'lot': False, 'valet': False}"
 "{'garage': True, 'street': False, 'validated': Tru

### 4.1.1. str type으로 dict이 선언되어 있는 칼럼 dict type으로 먼저 풀어줌

In [ ]:
preprocess_cols_index = [9,19,24,27,30,34,38]

In [ ]:
import ast

for col_index in preprocess_cols_index:
    attributes_df[attributes_df.columns[col_index]] = attributes_df[attributes_df.columns[col_index]].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else {})

attributes_df

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,BusinessParking,...,AcceptsInsurance,BestNights,BYOB,Corkage,BYOBCorkage,HairSpecializesIn,Open24Hours,RestaurantsCounterService,AgesAllowed,DietaryRestrictions
0,True,False,False,False,False,False,False,False,False,{},...,False,{},False,False,False,{},False,False,False,{}
1,False,True,False,False,False,False,False,False,False,{},...,False,{},False,False,False,{},False,False,False,{}
2,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,False,{},False,False,False,{},False,False,False,{}
3,False,False,True,1,False,True,False,True,u'free',"{'garage': False, 'street': True, 'validated':...",...,False,{},False,False,False,{},False,False,False,{}
4,False,True,True,False,False,True,False,False,False,"{'garage': None, 'street': None, 'validated': ...",...,False,{},False,False,False,{},False,False,False,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,3,False,False,False,False,False,{},...,False,{},False,False,False,{},False,False,False,{}
150342,False,True,True,2,False,False,False,False,u'no',"{'garage': False, 'street': False, 'validated'...",...,False,{},False,False,False,{},False,False,False,{}
150343,False,True,False,1,False,False,False,False,False,{},...,False,{},False,False,False,{},False,False,False,{}
150344,False,True,True,4,False,None,None,False,False,"{'garage': False, 'street': False, 'validated'...",...,False,{},False,False,False,{},False,False,False,{}


In [ ]:
# str type의 dict 값이 dict type으로 변환됨을 확인

for i in attributes_df.iloc[3]:
    print(i, type(i))

False <class 'bool'>
False <class 'bool'>
True <class 'bool'>
1 <class 'str'>
False <class 'bool'>
True <class 'bool'>
False <class 'bool'>
True <class 'bool'>
u'free' <class 'str'>
{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False} <class 'dict'>
False <class 'bool'>
False <class 'bool'>
False <class 'bool'>
False <class 'bool'>
False <class 'bool'>
False <class 'bool'>
u'none' <class 'str'>
False <class 'bool'>
False <class 'bool'>
{} <class 'dict'>
False <class 'bool'>
False <class 'bool'>
False <class 'bool'>
False <class 'bool'>
{} <class 'dict'>
False <class 'bool'>
False <class 'bool'>
{} <class 'dict'>
False <class 'numpy.bool_'>
False <class 'bool'>
{} <class 'dict'>
False <class 'bool'>
False <class 'bool'>
False <class 'bool'>
{} <class 'dict'>
False <class 'numpy.bool_'>
False <class 'numpy.bool_'>
False <class 'bool'>
{} <class 'dict'>


### 4.1.2. dict type 에서 key는 column으로 추가

In [ ]:
for col_index in preprocess_cols_index:
    tmp_df = pd.DataFrame(attributes_df[attributes_df.columns[col_index]].tolist())
    tmp_df = tmp_df.fillna(False)

    attributes_df = pd.concat([attributes_df,tmp_df], axis=1)

attributes_df = attributes_df.drop(attributes_df.columns[preprocess_cols_index], axis=1)
attributes_df

,ByAppointmentOnly,BusinessAcceptsCreditCards,BikeParking,RestaurantsPriceRange2,CoatCheck,RestaurantsTakeOut,RestaurantsDelivery,Caters,WiFi,WheelchairAccessible,...,kids,perms,asian,dairy-free,gluten-free,vegan,kosher,halal,soy-free,vegetarian
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,True,True,2,False,False,False,False,u'no',True,...,False,False,False,False,False,False,False,False,False,False
3,False,False,True,1,False,True,False,True,u'free',False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,True,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,3,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,True,True,2,False,False,False,False,u'no',False,...,False,False,False,False,False,False,False,False,False,False
150343,False,True,False,1,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,True,True,4,False,None,None,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 4.1.3 business_df 에 전처리된 attributes_df 를 concat

In [ ]:
business_df = business_df.drop(['attributes'], axis=1)
business_df = pd.concat([business_df, attributes_df], axis=1)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,kids,perms,asian,dairy-free,gluten-free,vegan,kosher,halal,soy-free,vegetarian
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,False,False,False,False,False,False,False,False,False,False
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,False,False,False,False,False,False,False,False,False,False
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,False,False,False,False,False,False,False,False,False,False
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,False,False,False,False,False,False,False,False,False,False
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,...,False,False,False,False,False,False,False,False,False,False
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,...,False,False,False,False,False,False,False,False,False,False
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,...,False,False,False,False,False,False,False,False,False,False
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,...,False,False,False,False,False,False,False,False,False,False


## 4.2. categories 칼럼 전처리
   - str type
   - 여러 카테고리에 대하여 콤마(,)로 정보가 들어있기 때문에 split 후 column으로 변환시킬 필요가 있음

In [ ]:
categories_df = pd.DataFrame(business_df['categories'])
categories_df

,categories
0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,"Shipping Centers, Local Services, Notaries, Ma..."
2,"Department Stores, Shopping, Fashion, Home & G..."
3,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,"Brewpubs, Breweries, Food"
...,...
150341,"Nail Salons, Beauty & Spas"
150342,"Pets, Nurseries & Gardening, Pet Stores, Hobby..."
150343,"Shopping, Jewelry, Piercing, Toy Stores, Beaut..."
150344,"Fitness/Exercise Equipment, Eyewear & Optician..."


In [ ]:
categories_df['categories'] = categories_df['categories'].apply(lambda x: [i.strip() for i in x.split(',')])

# 리스트를 더미 변수로 변환
categories_encoded = categories_df['categories'].str.join('|').str.get_dummies().astype(bool)

# 결과 확인
categories_encoded

,& Probates,3D Printing,ATV Rentals/Tours,Acai Bowls,Accessories,Accountants,Acne Treatment,Active Life,Acupuncture,Addiction Medicine,...,Wine Tasting Room,Wine Tours,Wineries,Women's Clothing,Workers Compensation Law,Wraps,Yelp Events,Yoga,Ziplining,Zoos
0,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150343,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 4.2.1. 가장 많은 카테고리 순으로 열 순서 재정렬

In [ ]:
import pandas as pd

# Assuming categories_encoded is your pandas DataFrame

# Calculate the count of True values in each column
true_counts = categories_encoded.sum()

# Sort the columns based on the count of True values in descending order
sorted_columns = true_counts.sort_values(ascending=False).index

# Reorder the DataFrame columns
categories_encoded = categories_encoded[sorted_columns]
categories_encoded

,Restaurants,Food,Shopping,Home Services,Beauty & Spas,Nightlife,Health & Medical,Local Services,Bars,Automotive,...,Roman,Dialysis Clinics,Lahmacun,Donburi,Vermouth Bars,Aviation Services,Props,Mohels,Faith-based Crisis Pregnancy Centers,Conveyor Belt Sushi
0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150343,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 4.2.2. column(카테고리)끼리 중복이 있나 검사
  - 없음

In [ ]:
import pandas as pd

# Assuming categories_encoded is your pandas DataFrame

# Calculate the count of True values in each column
true_counts = categories_encoded.sum()

# Sort the columns based on the count of True values in descending order
sorted_columns = true_counts.sort_values(ascending=False).index

# Remove columns with overlapping sorted values, keeping only the one with the smallest index
unique_columns = []
seen_columns = set()
for col in sorted_columns:
    if col not in seen_columns:
        unique_columns.append(col)
        seen_columns.add(col)

# Reorder the DataFrame columns
categories_encoded = categories_encoded[unique_columns]
categories_encoded

,Restaurants,Food,Shopping,Home Services,Beauty & Spas,Nightlife,Health & Medical,Local Services,Bars,Automotive,...,Paragliding,Parklets,Parent Cafes,Aircraft Dealers,Tonkatsu,Pita,Storefront Clinics,Sperm Clinic,Sport Equipment Hire,Conveyor Belt Sushi
0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150343,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 4.2.3. column(카테고리)가 포함되는 것이 있나 검사
  - 있다면 앞선 column만 남김
  - 11개 항목 삭제

In [ ]:
import pandas as pd

# Assuming categories_encoded is your pandas DataFrame

# Calculate the count of True values in each column
true_counts = categories_encoded.sum()

# Sort the columns based on the count of True values in descending order
sorted_columns = true_counts.sort_values(ascending=False).index

# Keep only the column with the smallest index when there is an overlap
unique_columns = []
seen_columns = set()
for col in sorted_columns:
    included = False
    for seen_col in seen_columns:
        if col in seen_col:
            included = True
            break
    if not included:
        unique_columns.append(col)
        seen_columns.add(col)

# Reorder the DataFrame columns
categories_encoded = categories_encoded[unique_columns]
categories_encoded

,Restaurants,Food,Shopping,Home Services,Beauty & Spas,Nightlife,Health & Medical,Local Services,Bars,Automotive,...,Dart Arenas,Roman,Dialysis Clinics,Beer Hall,Vermouth Bars,Aviation Services,Props,Mohels,Faith-based Crisis Pregnancy Centers,Conveyor Belt Sushi
0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150343,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# 레스토랑과 음식점 차이 확인 -> 49,105개가 다른걸로 봐서 포함관계는 아닌듯.

import pandas as pd

# Assuming categories_encoded is your pandas DataFrame
column1 = 'Restaurants'
column2 = 'Food'

# Calculate the number of differences between the two columns for each row
differences = (categories_encoded[column1] != categories_encoded[column2]).sum()

# Print the number of differences
differences

49105

### 4.2.4 모든 데이터가 최소 한개의 카테고리에 포함되게 하는 상위 n개 카테고리 지정
  - 우선 모든 값이 False인 row는 삭제(하지 않으면 상위 n개를 찾을 수 없음)
  - 반복문으로 찾기(10분 이상 소요)
  - 이진탐색으로 찾기(반복문에 비해 훨씬 효율적)

  - 453개

In [ ]:
categories_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Columns: 1300 entries, Restaurants to Conveyor Belt Sushi
dtypes: bool(1300)
memory usage: 186.4 MB


In [ ]:
# import pandas as pd

# # Assuming categories_encoded is your pandas DataFrame

# n = 1  # Initialize n to 1
# all_rows_satisfied = False

# while not all_rows_satisfied:
#     # Get the top n columns with the highest sum
#     top_columns = categories_encoded.sum().nlargest(n).index

#     # Check if each row has a True value in at least one of the top columns
#     all_rows_satisfied = categories_encoded[top_columns].any(axis=1).all()

#     if not all_rows_satisfied:
#         n += 1

# # Update the DataFrame to ensure each row has a True value in at least one of the top columns
# categories_encoded['AtLeastOneTrue'] = categories_encoded[top_columns].any(axis=1)

# # Print the updated DataFrame
# categories_encoded

In [ ]:
categories_encoded

,Restaurants,Food,Shopping,Home Services,Beauty & Spas,Nightlife,Health & Medical,Local Services,Bars,Automotive,...,Dart Arenas,Roman,Dialysis Clinics,Beer Hall,Vermouth Bars,Aviation Services,Props,Mohels,Faith-based Crisis Pregnancy Centers,Conveyor Belt Sushi
0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150343,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
import pandas as pd

# Assuming categories_encoded is your pandas DataFrame

# Binary search to find the smallest n that satisfies the condition
def binary_search(categories_encoded):
    low = 1
    high = len(categories_encoded.columns)

    while low < high:
        mid = (low + high) // 2

        # Get the top mid columns with the highest sum
        top_columns = categories_encoded.sum().nlargest(mid).index

        # Check if each row has a True value in at least one of the top columns
        all_rows_satisfied = categories_encoded[top_columns].any(axis=1).all()

        if all_rows_satisfied:
            high = mid
        else:
            low = mid + 1

    return low

# Remove rows where all values are False
categories_encoded_filtered = categories_encoded.loc[(categories_encoded == True).any(axis=1)]

# Find the smallest n that satisfies the condition
n = binary_search(categories_encoded_filtered)

# Get the top n columns with the highest sum
top_columns = categories_encoded_filtered.sum().nlargest(n).index.tolist()

# Print the list of top columns
len(top_columns)

453

In [ ]:
# Show only the top_columns in categories_encoded DataFrame
categories_encoded_top_columns = categories_encoded[top_columns]

# Print the categories_encoded DataFrame with only the top_columns
categories_encoded_top_columns

,Restaurants,Food,Shopping,Home Services,Beauty & Spas,Nightlife,Health & Medical,Local Services,Bars,Automotive,...,Snow Removal,Recycling Center,Television Service Providers,Home Window Tinting,Barre Classes,Fishing,Home Organization,Irish Pub,Lawn Services,Mass Media
0,False,False,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150342,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150343,False,False,True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
150344,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### 4.2.5. business_df 에 categories_encoded_top_columns을 concat

In [ ]:
business_df = business_df.drop(['categories'], axis=1)
business_df = pd.concat([business_df, categories_encoded_top_columns], axis=1)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Snow Removal,Recycling Center,Television Service Providers,Home Window Tinting,Barre Classes,Fishing,Home Organization,Irish Pub,Lawn Services,Mass Media
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,False,False,False,False,False,False,False,False,False,False
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,False,False,False,False,False,False,False,False,False,False
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,False,False,False,False,False,False,False,False,False,False
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,False,False,False,False,False,False,False,False,False,False
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,...,False,False,False,False,False,False,False,False,False,False
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,...,False,False,False,False,False,False,False,False,False,False
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,...,False,False,False,False,False,False,False,False,False,False
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,...,False,False,False,False,False,False,False,False,False,False


## 4.3. hours 칼럼 전처리

In [ ]:
hours_df = pd.DataFrame(business_df['hours'].tolist())
hours_df

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,NaN
2,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,NaN,NaN,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
...,...,...,...,...,...,...,...
150341,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-17:30,11:0-17:0
150342,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:0-17:0,0:0-16:0
150343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
150344,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-17:0,10:0-17:0


In [ ]:
hours_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150346 entries, 0 to 150345
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Monday     114474 non-null  object
 1   Tuesday    120631 non-null  object
 2   Wednesday  123771 non-null  object
 3   Thursday   125198 non-null  object
 4   Friday     124999 non-null  object
 5   Saturday   110770 non-null  object
 6   Sunday     81172 non-null   object
dtypes: object(7)
memory usage: 8.0+ MB


In [ ]:
for i in hours_df.iloc[1]:
    print(i, type(i))

0:0-0:0 <class 'str'>
8:0-18:30 <class 'str'>
8:0-18:30 <class 'str'>
8:0-18:30 <class 'str'>
8:0-18:30 <class 'str'>
8:0-14:0 <class 'str'>
nan <class 'float'>


### 4.3.1. 결측치 처리
  - row의 모든 값이 결측치 인 경우 정보제공 x -> 'no information'으로 대체
  - row의 일부 column값이 결측치 인 경우 -> 가게 휴일이므로 'closed' 대체

In [ ]:
import pandas as pd
import numpy as np

# Assuming hours_df is your pandas DataFrame

# Function to replace missing values based on conditions
def replace_missing_values(row):
    # Check if all values in the row are NaN
    if row.isnull().all():
        return pd.Series(['no information'] * len(row), index=row.index)
    else:
        return row.fillna('closed')

# Apply the replace_missing_values function row-wise and convert back to DataFrame
hours_df_updated = hours_df.apply(replace_missing_values, axis=1).apply(pd.Series)

# Print the updated DataFrame
hours_df_updated

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,no information,no information,no information,no information,no information,no information,no information
1,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,closed
2,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,closed,closed,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
...,...,...,...,...,...,...,...
150341,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-17:30,11:0-17:0
150342,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:0-17:0,0:0-16:0
150343,no information,no information,no information,no information,no information,no information,no information
150344,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-17:0,10:0-17:0


### 4.3.2. 기존 business_df와 hours_df_updated concat

In [ ]:
business_df = business_df.drop(['hours'], axis=1)
business_df = pd.concat([business_df,hours_df_updated ], axis=1)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,Irish Pub,Lawn Services,Mass Media,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,...,False,False,False,no information,no information,no information,no information,no information,no information,no information
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,...,False,False,False,0:0-0:0,8:0-18:30,8:0-18:30,8:0-18:30,8:0-18:30,8:0-14:0,closed
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,...,False,False,False,8:0-22:0,8:0-22:0,8:0-22:0,8:0-22:0,8:0-23:0,8:0-23:0,8:0-22:0
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,False,False,False,7:0-20:0,7:0-20:0,7:0-20:0,7:0-20:0,7:0-21:0,7:0-21:0,7:0-21:0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,False,False,False,closed,closed,14:0-22:0,16:0-22:0,12:0-22:0,12:0-22:0,12:0-18:0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,...,False,False,False,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-19:30,10:0-17:30,11:0-17:0
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,...,False,False,False,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:30-17:30,9:0-17:0,0:0-16:0
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,...,False,False,False,no information,no information,no information,no information,no information,no information,no information
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,...,False,False,False,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-17:0,10:0-17:0


# 5. 최종 데이터셋 구축(business_df 와 total_df concat)

In [ ]:
import pandas as pd

# Assuming review_df and user_grouped_df are your pandas DataFrames

# Perform a left merge on 'user_id' column
total_df = pd.merge(total_df, business_df, on='business_id', how='left')

# Print the resulting DataFrame
total_df

,review_id,user_id,business_id,stars_x,useful,funny,cool,text,date,mean,...,Irish Pub,Lawn Services,Mass Media,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,4.142857,...,False,False,False,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,4.125000,...,False,False,False,6:30-20:30,6:30-20:30,6:0-20:30,6:30-20:30,6:0-19:30,7:30-12:30,9:0-13:0
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,4.448980,...,False,False,False,no information,no information,no information,no information,no information,no information,no information
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,4.800000,...,False,False,False,closed,11:0-21:0,11:0-21:0,11:0-21:0,11:0-22:0,11:0-22:0,11:0-21:0
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,3.008264,...,False,False,False,0:0-0:0,closed,closed,closed,11:0-17:0,closed,11:0-11:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20,3.646643,...,False,False,False,0:0-0:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-15:0,closed
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10,5.000000,...,False,False,False,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30,3.936306,...,False,False,False,0:0-0:0,6:0-22:0,6:0-22:0,6:0-18:0,9:0-20:0,6:0-22:0,7:0-20:0
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,4.345238,...,False,False,False,0:0-0:0,11:0-21:0,11:0-21:0,11:0-21:0,11:0-20:0,12:0-20:0,12:0-20:0


In [ ]:
# Assuming total_df is your pandas DataFrame

# Rename the 'star' column to 'review_star'
total_df = total_df.rename(columns={'stars_x': 'review_star', 'stars_y':'business_star'})

# Print the updated DataFrame
total_df

,review_id,user_id,business_id,review_star,useful,funny,cool,text,date,mean,...,Irish Pub,Lawn Services,Mass Media,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11,4.142857,...,False,False,False,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0,7:30-15:0
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18,4.125000,...,False,False,False,6:30-20:30,6:30-20:30,6:0-20:30,6:30-20:30,6:0-19:30,7:30-12:30,9:0-13:0
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30,4.448980,...,False,False,False,no information,no information,no information,no information,no information,no information,no information
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03,4.800000,...,False,False,False,closed,11:0-21:0,11:0-21:0,11:0-21:0,11:0-22:0,11:0-22:0,11:0-21:0
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15,3.008264,...,False,False,False,0:0-0:0,closed,closed,closed,11:0-17:0,closed,11:0-11:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20,3.646643,...,False,False,False,0:0-0:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-18:0,9:0-15:0,closed
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10,5.000000,...,False,False,False,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0,9:0-17:0
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30,3.936306,...,False,False,False,0:0-0:0,6:0-22:0,6:0-22:0,6:0-18:0,9:0-20:0,6:0-22:0,7:0-20:0
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,4.345238,...,False,False,False,0:0-0:0,11:0-21:0,11:0-21:0,11:0-21:0,11:0-20:0,12:0-20:0,12:0-20:0


# 6. 카테고리 별 data split

In [ ]:
# 복사해서 따로 관리
total_dataset = total_df.copy()

## 6.1: 팁을 주지 않아도 되는 categories 제거

In [ ]:
remove_categories= ['Shopping', 'Health & Medical', 'Automotive',
       'American (Traditional)', 'Fast Food', 'Fashion', 'Burgers',
       'Auto Repair', 'Specialty Food', 'Doctors', 'Pets', 'Real Estate',
       'Professional Services', 'Grocery', 'Pet Services', 'Dentists',
       'Tires', 'Oil Change Stations', 'Contractors', 'General Dentistry',
       'Education', 'Auto Parts & Supplies', 'Apartments',
       'Convenience Stores', 'Home Decor', 'Gyms', 'Car Dealers',
       'Sporting Goods', 'Accessories', 'Drugstores', 'Local Flavor',
       'Furniture Stores', 'Cosmetics & Beauty Supply', 'Jewelry',
       'Financial Services', 'Trainers', 'Arts & Crafts',
       'Department Stores', 'Electronics', 'Veterinarians', 'Pet Sitting',
       "Men's Clothing", 'Food Trucks', 'Body Shops',
       'Public Services & Government', 'Heating & Air Conditioning/HVAC',
       'IT Services & Computer Repair', 'Laundry Services',
       'Medical Centers', 'Optometrists', 'Plumbing', 'Music Venues',
       'Dry Cleaning & Laundry', 'Party & Event Planning', 'Gas Stations',
       'Pet Stores', 'Shoe Stores', 'Eyewear & Opticians', 'Parks',
       'Books', 'Music & Video', 'Mags', 'Car Wash', 'Florists',
       'Building Supplies', 'Mobile Phones', 'Movers',
       'Specialty Schools', 'Auto Detailing', 'Landscaping',
       'Nurseries & Gardening', 'Appliances', 'Discount Store',
       'Transportation', 'Home Cleaning', 'Tours', 'Sewing & Alterations',
       'Appliances & Repair', 'Real Estate Services', 'Gift Shops',
       'Self Storage', 'Shipping Centers', 'Hardware Stores',
       'Printing Services', 'Art Galleries', 'Kitchen & Bath',
       'Property Management', 'Vintage & Consignment', 'Used',
       'Oral Surgeons', 'Banks & Credit Unions', 'Auto Glass Services',
       'Chiropractors', 'Endodontists', 'Towing', 'Tobacco Shops',
       'Car Rental', 'Tanning', 'Orthodontists', 'Transmission Repair',
       'Antiques', 'Mattresses', 'Hair Extensions', 'Flooring',
       'Thrift Stores', 'Tattoo', 'Health Markets', 'Performing Arts',
       'Sports Wear', 'Real Estate Agents', 'Pet Training',
       'Physical Therapy', 'Tree Services',
       'Water Heater Installation/Repair', 'Bridal', 'Meat Shops',
       'Hobby Shops', 'Urgent Care', 'Fruits & Veggies',
       'Pediatric Dentists', 'Carpet Cleaning', 'Mobile Phone Repair',
       'Toy Stores', 'Notaries', 'Interior Design', 'Photographers',
       'Electronics Repair', 'Community Service/Non-Profit', 'Pharmacy',
       'Pest Control', 'Family Practice', 'Farmers Market', 'Golf',
       'Electricians', 'Vape Shops', 'Roofing',
       'Mobile Phone Accessories', 'Diagnostic Services',
       'Photography Stores & Services', 'Periodontists', 'Bikes',
       'Blow Dry/Out Services', 'Eyebrow Services', 'Dance Clubs',
       'Windows Installation', 'Painters', "Children's Clothing",
       'Festivals', 'Weight Loss Centers', 'Nutritionists',
       'Used Car Dealers', 'Food Stands', 'Cupcakes', 'Ophthalmologists',
       'Child Care & Day Care', 'Bookstores', 'Museums', 'Insurance',
       'Junk Removal & Hauling', 'Telecommunications', 'Hospitals',
       'Arcades', 'Office Cleaning', 'Outdoor Gear', 'Kids Activities',
       'Cosmetic Surgeons', 'Lawyers', 'Masonry/Concrete',
       'Mortgage Brokers', 'Cinema', 'Session Photography',
       'Street Vendors', 'Handyman', 'Internet Service Providers',
       'Candy Stores', 'Damage Restoration',
       'Windshield Installation & Repair', 'Jewelry Repair', 'Preschools',
       'Chocolatiers & Shops', 'Computers', 'Mailbox Centers',
       'Boot Camps', 'Seafood Markets', 'Lighting Fixtures & Equipment',
       'Dermatologists', 'Watches', 'Post Offices', 'Karaoke',
       'Bed & Breakfast', 'Keys & Locksmiths', 'Gutter Services',
       'Couriers & Delivery Services', 'Sports Clubs',
       'Auto Customization', 'Art Supplies',
       'Obstetricians & Gynecologists', 'Party Supplies', 'Carpeting',
       'Landmarks & Historical Buildings', 'Shopping Centers',
       'Bike Rentals', 'Pilates', 'Organic Stores', 'Sports Medicine',
       'Security Systems', 'Spanish', 'Religious Organizations',
       'Home Inspectors', 'Formal Wear', 'Amusement Parks',
       'Swimming Pools', 'Coffee Roasteries',
       'Musical Instruments & Teachers', 'Door Sales/Installation',
       'Wholesale Stores', 'Bike Repair/Maintenance', 'Shaved Ice',
       'Laboratory Testing', 'Air Duct Cleaning', 'Playgrounds',
       'Shades & Blinds', 'Martial Arts', 'Office Equipment',
       'Hot Tub & Pool', 'Floral Designers', 'Shoe Repair', 'Gardeners',
       'Vitamins & Supplements', 'Cards & Stationery', 'Pediatricians',
       'Car Stereo Installation', 'Guns & Ammo', 'Recreation Centers',
       'Parking', 'Baby Gear & Furniture', 'Art Classes', 'Watch Repair',
       'Landscape Architects', 'Graphic Design', 'Tiling', 'Summer Camps',
       'Window Washing', 'Music & DVDs', 'Car Window Tinting',
       'Motorcycle Dealers', 'Orthopedists', 'Counseling & Mental Health',
       'Elementary Schools', 'Rehabilitation Center', 'Glass & Mirrors',
       'Naturopathic/Holistic', 'Smog Check Stations',
       'Adult Entertainment', 'Churches', 'Internal Medicine',
       'Cabinetry', 'Hiking', 'Poke', 'Pool Cleaners', 'Irrigation',
       'Libraries', 'Truck Rental', 'International Grocery', 'Signmaking',
       'Motorcycle Repair', 'Interval Training Gyms', 'Dance Studios',
       'Reiki', 'Imported Food', 'Lingerie', 'Head Shops',
       'Fences & Gates', 'Laundromat', 'Auto Insurance',
       'Medical Supplies', 'Framing', 'Bowling',
       'Party Equipment Rentals',
       'Laser Eye Surgery/Lasik', 'Videos & Video Game Rental',
       'Outlet Stores', 'Butcher', 'Siding', 'Financial Advising',
       'Colleges & Universities', 'Diagnostic Imaging', 'Fabric Stores',
       'Auto Loan Providers', 'Escape Games',
       'Musical Instrument Services', 'Garage Door Services',
       'Pool Halls', 'Packing Supplies', 'Dog Parks', 'Pet Adoption',
       'Funeral Services & Cemeteries', 'Furniture Reupholstery',
       'Roadside Assistance', 'Recycling Center',
       'Television Service Providers', 'Home Window Tinting',
       'Barre Classes', 'Home Organization', 'Lawn Services',
       'Mass Media']

In [ ]:
total_dataset = total_dataset[~total_dataset[remove_categories].any(axis=1)]

print("원래 dataset size: ", total_df.shape)
print("split 후 dataset size : ", total_dataset.shape)

원래 dataset size:  (6990280, 563)
split 후 dataset size :  (3494866, 563)


## 6.2 레스토랑에서 priceRange2 가 낮은 categories
- default : $20
- file name : total_df_01_20

In [ ]:
temp_df_01 = total_dataset[total_dataset['Restaurants'] == True]
total_df_01_20 = temp_df_01[(temp_df_01['RestaurantsPriceRange2'] == '1') |  (temp_df_01['RestaurantsPriceRange2'] == '2')]

# fiel 저장 경로
# file_path = './TM_split_dataset/total_df_01_res_20.csv'

# file 저장
#total_df_01_20.to_csv(file_path, index=False)

print("split 전 dataset size: ", total_dataset.shape)

# 저장 후 total_dataset에서 drop
total_dataset = total_dataset.drop(total_df_01_20.index)

print("split 후 dataset size : ", total_dataset.shape)

split 전 dataset size:  (3494866, 563)
split 후 dataset size :  (972721, 563)


## 6.3 Beauty & Spas
- Beaty & Spas 이면서 Resutaurant == False
- default : $20
- file name : total_df_02_beaty_20.csv

In [ ]:
total_df_02_20 = total_dataset[(total_dataset['Beauty & Spas'] == True ) & (total_dataset['Restaurants'] == False ) ]

# CSV 파일로 저장
#file_path = './TM_split_dataset/total_df_02_beaty_20.csv'
#total_df_02_20.to_csv(file_path, index=False)

print("split 전 dataset size: ", total_dataset.shape)

# 저장 후 total_dataset에서 drop
total_dataset = total_dataset.drop(total_df_02_20.index)

print("split 후 dataset size : ", total_dataset.shape)

split 전 dataset size:  (972721, 563)
split 후 dataset size :  (759824, 563)


## 6.4 지정 가격이 있는 categories 삭제
- ex) HOTEL은 벨보이한테 $1

In [ ]:
remove_categories2= ['Hotels & Travel','Home & Garden','Wineries','Airport Shuttles', 'Jazz & Blues']

print("split 전 dataset size: ", total_dataset.shape)

total_dataset = total_dataset[~total_dataset[remove_categories2].any(axis=1)]

print("split 후 dataset size : ", total_dataset.shape)

split 전 dataset size:  (759824, 563)
split 후 dataset size :  (606574, 563)


## 6.5 Restaurant 중에 PriceRange가 높은 애들
- default : $ 22
- file name : total_df_03_res_22.csv

In [ ]:
temp_df_03 = total_dataset[total_dataset['Restaurants'] == True]
total_df_03_22 = temp_df_03[(temp_df_03['RestaurantsPriceRange2'] == '3') |  (temp_df_03['RestaurantsPriceRange2'] == '4')]

# CSV 파일로 저장
#file_path = './TM_split_dataset/total_df_03_res_22.csv'
#total_df_03_22.to_csv(file_path, index=False)

print("split 전 dataset size: ", total_dataset.shape)

# 저장했으니깐 total_dataset에서 drop!
total_dataset = total_dataset.drop(total_df_03_22.index)

print("split 후 dataset size : ", total_dataset.shape)

split 전 dataset size:  (606574, 563)
split 후 dataset size :  (387772, 563)


## 6.6 Categories 중 10%~20% 팁을 주는 categories
- default : $10
- file name : total_df_04_10.csv

In [ ]:
# 해당되는 categories name
dollar_10 = ['Wedding Planning', 'Tapas Bars']
total_df_04_10 = total_dataset[total_dataset[dollar_10].any(axis=1)]

# CSV 파일로 저장
#file_path = './TM_split_dataset/total_df_04_10.csv'
#total_df_04_10.to_csv(file_path, index=False)

print("split 전 dataset size: ", total_dataset.shape)

# 저장했으니깐 total_dataset에서 삭제
total_dataset = total_dataset.drop(total_df_04_10.index)

print("split 후 dataset size : ", total_dataset.shape)

split 전 dataset size:  (387772, 563)
split 후 dataset size :  (385741, 563)


## 6.7 Categories 중
- default : $15
- file name : total_df_05_15.csv

In [ ]:
dollar_15 = ['Pizza', 'Coffee & Tea', 'Breakfast & Brunch', 'Desserts',
       'Chinese', 'Bakeries', 'Salad', 'Chicken Wings', 'Cafes',
       'Ice Cream & Frozen Yogurt', 'Wine & Spirits', 'Beer', 'Delis',
       "Women's Clothing", 'Sports Bars', 'Flowers & Gifts', 'Pubs',
       'Juice Bars & Smoothies', 'Barbers', 'Cosmetic Dentists',
       'Pet Groomers', 'Eyelash Service', 'Donuts', 'Soup', 'Yoga',
       "Men's Hair Salons", 'Chicken Shop', 'Medical Spas', 'Hot Dogs',
       'Bagels', 'Ethnic Food', 'Dive Bars', 'Buffets', 'Cheesesteaks',
       'Caribbean', 'Middle Eastern', 'Pet Boarding', 'Taxis',
       'Hookah Bars', 'Pakistani', 'Tea Rooms', 'Ramen', 'Irish Pub']

total_df_05_15 = total_dataset[total_dataset[dollar_15].any(axis=1)]

# CSV 파일로 저장
#file_path = './TM_split_dataset/total_df_05_15.csv'
#total_df_05_15.to_csv(file_path, index=False)

print("split 전 dataset size: ", total_dataset.shape)

# 저장했으니깐 total_dataset에서 삭제
total_dataset = total_dataset.drop(total_df_05_15.index)

print("split 후 dataset size : ", total_dataset.shape)

split 전 dataset size:  (385741, 563)
split 후 dataset size :  (126616, 563)


## 6.8 Catrgories 중
- default : $18
- file name : total_df_06_18.csv

In [ ]:
dollar_18 =['Home Services', 'Local Services', 'Event Planning & Services',
       'Sandwiches', 'Active Life', 'Arts & Entertainment',
       'Fitness & Instruction', 'Acupuncture', 'Reflexology',
       'Laser Hair Removal', 'Permanent Makeup', 'Piercing',
       'Spray Tanning', 'Event Photography', 'Pressure Washers', 'Wraps',
       'Wheel & Rim Repair', 'Pool & Hot Tub Service', 'Custom Cakes',
       'Boating', 'Dog Walkers', 'Travel Services', 'Packing Services',
       'Canadian (New)', 'Traditional Chinese Medicine', 'Resorts',
       'Personal Shopping', 'Snow Removal']

total_df_06_18 = total_dataset[total_dataset[dollar_18].any(axis=1)]

# CSV 파일로 저장
#file_path = './TM_split_dataset/total_df_06_18.csv'
#total_df_06_18.to_csv(file_path, index=False)

print("split 전 dataset size: ", total_dataset.shape)

# 저장했으니깐 total_dataset에서 삭제
total_dataset = total_dataset.drop(total_df_06_18.index)

print("split 후 dataset size : ", total_dataset.shape)

split 전 dataset size:  (126616, 563)
split 후 dataset size :  (92810, 563)


## 6.9 Categories 중
- default : $20
- file name : total_df_07_20.csv

In [ ]:
dollar_20 = ['Restaurants', 'Food', 'Beauty & Spas', 'Nightlife', 'Bars',
       'American (New)', 'Hair Salons', 'Nail Salons', 'Mexican',
       'Italian', 'Seafood', 'Hair Removal', 'Caterers', 'Skin Care',
       'Venues & Event Spaces', 'Waxing', 'Massage', 'Day Spas',
       'Japanese', 'Cocktail Bars', 'Sushi Bars', 'Barbeque',
       'Asian Fusion', 'Steakhouses', 'Diners', 'Hair Stylists',
       'Massage Therapy', 'Mediterranean', 'Lounges', 'Wine Bars',
       'Vegetarian', 'Food Delivery Services', 'Tacos', 'Southern',
       'Makeup Artists', 'Thai', 'Cajun/Creole', 'Tex-Mex', 'Beer Bar',
       'Breweries', 'Vegan', 'Vietnamese', 'Indian', 'Latin American',
       'Greek', 'Gluten-Free', 'Comfort Food', 'Soul Food', 'Gastropubs',
       'French', 'Korean', 'Bubble Tea', 'Tapas/Small Plates', 'Noodles',
       'Halal', 'Nail Technicians', 'Limos', 'Cuban', 'Acai Bowls',
       'Irish', 'Fish & Chips', 'Gelato', 'Waffles', 'Brewpubs',
       'Hawaiian', 'Beer Gardens', 'Creperies', 'Fishing', 'Pasta Shops']

total_df_07_20 = total_dataset[total_dataset[dollar_20].any(axis=1)]

# CSV 파일로 저장
#file_path = './TM_split_dataset/total_df_07_20.csv'
#total_df_07_20.to_csv(file_path, index=False)

print("split 전 dataset size: ", total_dataset.shape)

# 저장했으니깐 total_dataset에서 삭제
total_dataset = total_dataset.drop(total_df_07_20.index)

split 전 dataset size:  (92810, 563)


# 7. 카테고리별 splited dataset 저장

## 7.1. splited dataset 용량초과 문제 해결
- 30000개 이상 데이터셋은 3만개 샘플링 진행(팀원 개인 노트북 or Colab에서 진행 시 RAM 용량 초과로 전체 데이터셋 활용 불가)
- summarize 모델 input sequence 제한으로 인해 일정 토큰 이상의 text 행은 삭제

In [ ]:
total_df_01_20['word_count'] = total_df_01_20['text'].str.split().str.len()
total_df_01_20 = total_df_01_20[total_df_01_20['word_count'] <= 300]
total_df_01_20 = total_df_01_20.drop('word_count', axis=1).reset_index(drop=True)

total_df_01_20.shape

C:\Users\김동준\AppData\Local\Temp\ipykernel_16216\2552691141.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df_01_20['word_count'] = total_df_01_20['text'].str.split().str.len()


(2427464, 563)

In [ ]:
total_df_02_20['word_count'] = total_df_02_20['text'].str.split().str.len()
total_df_02_20 = total_df_02_20[total_df_02_20['word_count'] <= 300]
total_df_02_20 = total_df_02_20.drop('word_count', axis=1).reset_index(drop=True)

total_df_02_20.shape

(204027, 563)

In [ ]:
total_df_03_22['word_count'] = total_df_03_22['text'].str.split().str.len()
total_df_03_22 = total_df_03_22[total_df_03_22['word_count'] <= 300]
total_df_03_22 = total_df_03_22.drop('word_count', axis=1).reset_index(drop=True)

total_df_03_22.shape

C:\Users\김동준\AppData\Local\Temp\ipykernel_16216\2402878566.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_df_03_22['word_count'] = total_df_03_22['text'].str.split().str.len()


(200861, 563)

In [ ]:
total_df_04_10['word_count'] = total_df_04_10['text'].str.split().str.len()
total_df_04_10 = total_df_04_10[total_df_04_10['word_count'] <= 300]
total_df_04_10 = total_df_04_10.drop('word_count', axis=1).reset_index(drop=True)

total_df_04_10.shape

(1851, 563)

In [ ]:
total_df_05_15['word_count'] = total_df_05_15['text'].str.split().str.len()
total_df_05_15 = total_df_05_15[total_df_05_15['word_count'] <= 300]
total_df_05_15 = total_df_05_15.drop('word_count', axis=1).reset_index(drop=True)

total_df_05_15.shape

(252716, 563)

In [ ]:
total_df_06_18['word_count'] = total_df_06_18['text'].str.split().str.len()
total_df_06_18 = total_df_06_18[total_df_06_18['word_count'] <= 300]
total_df_06_18 = total_df_06_18.drop('word_count', axis=1).reset_index(drop=True)

total_df_06_18.shape

(31721, 563)

In [ ]:
total_df_07_20['word_count'] = total_df_07_20['text'].str.split().str.len()
total_df_07_20 = total_df_07_20[total_df_07_20['word_count'] <= 300]
total_df_07_20 = total_df_07_20.drop('word_count', axis=1).reset_index(drop=True)

total_df_07_20.shape

(88803, 563)

In [ ]:
sample_total_df_01_20 = total_df_01_20.sample(n=30000, random_state=42).reset_index(drop=True)
sample_total_df_01_20.shape

(30000, 563)

In [ ]:
sample_total_df_02_20 = total_df_02_20.sample(n=30000, random_state=42).reset_index(drop=True)
sample_total_df_02_20.shape

(30000, 563)

In [ ]:
sample_total_df_03_22 = total_df_03_22.sample(n=30000, random_state=42).reset_index(drop=True)
sample_total_df_03_22.shape

(30000, 563)

In [ ]:
# 4는 3만개 이하
# sample_total_df_04_10 = total_df_04_10.sample(n=30000, random_state=42).reset_index(drop=True)
# sample_total_df_04_10.shape

In [ ]:
sample_total_df_05_15 = total_df_05_15.sample(n=30000, random_state=42).reset_index(drop=True)
sample_total_df_05_15.shape

(30000, 563)

In [ ]:
sample_total_df_06_18 = total_df_06_18.sample(n=30000, random_state=42).reset_index(drop=True)
sample_total_df_06_18.shape

(30000, 563)

In [ ]:
sample_total_df_07_20 = total_df_07_20.sample(n=30000, random_state=42).reset_index(drop=True)
sample_total_df_07_20.shape

(30000, 563)

In [ ]:
#CSV 파일로 저장
file_path = './splited_dataset/'

sample_total_df_01_20.to_csv(file_path + 'sample_total_df_01_res_20.csv')
sample_total_df_02_20.to_csv(file_path + 'sample_total_df_02_beaty_20.csv')
sample_total_df_03_22.to_csv(file_path + 'sample_total_df_03_res_22.csv')
total_df_04_10.to_csv(file_path + 'sample_total_df_04_10.csv')
sample_total_df_05_15.to_csv(file_path + 'sample_total_df_05_15.csv')
sample_total_df_06_18.to_csv(file_path + 'sample_total_df_06_18.csv')
sample_total_df_07_20.to_csv(file_path + 'sample_total_df_07_20.csv')